In [1]:
from __future__ import annotations

%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path

import numpy as np

from wufam.data.prepare_data import read_kf_data
from wufam.config.trading_config import TradingConfig
from wufam.dataset import Dataset
from wufam.strategies.base_strategy import BaseStrategy
from wufam.backtest.rolling_backtest import run_rolling_backtest
from wufam.metrics.metrics import calc_sharpe

PATH = Path("../data/kf_data")
START = "1970-01-01"
END = "2024-12-31"
DATASET = Dataset.BM_6_D
FACTORS_DATASET = Dataset.FACTORS_D
WEIGHTING = "value_weighted"
FACTOR_ANNUALIZE = 12

## 1.1

In [3]:
portfolios_total_r, portfolios_xs_r, factors_df, rf = read_kf_data(
    portfolios_filename=PATH / DATASET,
    factors_filename=PATH / FACTORS_DATASET,
    start_date=START,
    end_date=END,
    weighting=WEIGHTING,
)

In [4]:
assert (
    portfolios_total_r.shape[0]
    == portfolios_xs_r.shape[0]
    == factors_df.shape[0]
    == rf.shape[0]
)

## 1.2

In [5]:
from wufam.strategies.heuristics.equally_weighted import EWStrategy
from wufam.strategies.estimated.mean_var import MeanVariance
from wufam.strategies.estimated.min_var import MinVariance

trading_config = TradingConfig(total_exposure=1)

ew_strategy = EWStrategy()

In [6]:
from wufam.estimation.mean.sample_mu_estimator import SampleMuEstimator
from wufam.estimation.covariance.sample_cov_estimator import SampleCovEstimator

mv_strategy = MeanVariance(
    mu_estimator=SampleMuEstimator(),
    cov_estimator=SampleCovEstimator(),
    trading_config=trading_config,
    window_size=None,
)

In [7]:
from wufam.estimation.covariance.shrinkage.lw_cv_cov_estimator import (
    LedoitWolfCVCovEstimator,
)

# trading_config_min_var = TradingConfig(total_exposure=1, min_exposure=0.0)

min_var_strategy = MinVariance(
    cov_estimator=LedoitWolfCVCovEstimator(alphas=np.linspace(0.0, 1.0, 100)),
    # cov_estimator=LedoitWolfCVCovEstimator(),
    # trading_config=trading_config_min_var,
    trading_config=trading_config,
    window_size=None,
)

In [8]:
def evaluate_strategy(strategy: BaseStrategy) -> tuple[float, float]:
    total_r, turnover = run_rolling_backtest(
        strategy=strategy,
        excess_returns=portfolios_total_r,
        rf=rf,
        freq="M",
        trading_lag=1,
    )

    sr = calc_sharpe(
        strategy_total_r=total_r,
        rf_rate=rf,
        factor_annualize=FACTOR_ANNUALIZE,
    )

    return sr, turnover.mean()

In [9]:
evaluate_strategy(ew_strategy)

Optimizing Strategy:: 100%|██████████| 659/659 [00:00<00:00, 2239.78it/s]


(0.14435324790863463, np.float64(0.015230586766937925))

In [10]:
evaluate_strategy(mv_strategy)

Optimizing Strategy:: 100%|██████████| 659/659 [00:00<00:00, 832.51it/s]


(0.2354226967920204, np.float64(0.5621819355312848))

In [11]:
evaluate_strategy(min_var_strategy)

Optimizing Strategy:: 100%|██████████| 659/659 [02:58<00:00,  3.70it/s]


(0.2515069939365704, np.float64(0.1356462097176127))